In [10]:
import carla
import random
import math
from collections import defaultdict
import lib_spawn as spb
from importlib import reload
reload(spb) #per refreshare modifiche import

# Tolleranza per considerare i waypoints sullo stesso asse X o Y
tolerance = 2
#contatore di tutte le coppe di wp
pair_counter = 0
pair_indexer = defaultdict(int)

def p_wp(w):
  return f"({round(w.transform.location.x,3)}) - ({round(w.transform.location.y,3)})"

#lista di adiacenza di coppie di waypoints avendi il secondo waypoint in comune
#significa che hanno punti di partenza differenti ma si intersecano nello stesso waypoint 
def group_pairs_by_second_waypoint(waypoint_pairs):
    
    # Dizionario (mappa) per raggruppare le coppie in base alle coordinate (x, y) del secondo waypoint di ciascuna coppia
    coord_groups = defaultdict(list)
    
    # Raggruppa le coppie in base alle coordinate x e y del secondo waypoint
    for wp1, wp2 in waypoint_pairs:
        #la chiave è rappresentata dalle coordinate del secondo wp
        coord_wp2 = (round(wp2.transform.location.x, 3), round(wp2.transform.location.y, 3))
        coord_groups[coord_wp2].append((wp1, wp2))
    
    # Stampa i gruppi di coppie per ciascun secondo waypoint
    for wp2_coords, pairs in coord_groups.items():
        if len(pairs) > 1:
            print(f"Le coordinate {wp2_coords} accomunano le seguenti coppie:")
            for pair in pairs:
                print(p_wp(pair[0]) + ", " + p_wp(pair[1]))
            print()
        
    return coord_groups

#colora il percorso tra due waypoint, bianco se i punto sono allineati sulla stessa trada, rosso se c'è una curva
def draw_route(wp1, wp2, symbol, r, g, b):
    distance = 3.0  # Distanza tra i waypoint intermedi in metri
    symbol = str(symbol)

    wp1 = wp1.next(distance)[0]
    
    # Disegna simboli lungo il percorso fino al waypoint di destinazione
    while wp1.transform.location.distance(wp2.transform.location) > distance:
        # Disegna il simbolo nel waypoint corrente
        world.debug.draw_string(wp1.transform.location, symbol, draw_shadow=False,
                                color=carla.Color(r, g, b), life_time=60.0, persistent_lines=True)
        # Vai al prossimo waypoint
        wp1 = wp1.next(distance)[0]


#disegna due x rosse sui wp
def draw_couple(idx, wp1, wp2):
    # Disegna una 'X' su wp1 con il numero progressivo
    world.debug.draw_string(wp1.transform.location, f"X {idx}", 
                            draw_shadow=True, 
                            color=carla.Color(r=0, g=255, b=0), 
                            life_time=60.0, 
                            persistent_lines=True)

    # Disegna una 'X' su wp2 con lo stesso numero progressivo
    world.debug.draw_string(wp2.transform.location, f"X {idx}", 
                            draw_shadow=True, 
                            color=carla.Color(r=255, g=0, b=0), 
                            life_time=60.0, 
                            persistent_lines=True)

# Funzione che disegna il percorso tra ogni coppia di wp presenti nella lista in input
def draw_intersection(pairs):

    global pair_counter #per modificare variabile globale
    
    for (wp1, wp2) in pairs:
       
        # Ottieni le coordinate x e y dei waypoints
        x1, y1 = wp1.transform.location.x, wp1.transform.location.y
        x2, y2 = wp2.transform.location.x, wp2.transform.location.y

        dx = abs(x1 - x2)
        dy = abs(y1 - y2)
        
        # Controllo se sono sullo stesso asse x o y
        if dx <= tolerance or  dy <= tolerance:
            #print(f"Pair {idx}: I waypoints sono allineati ({wp1.transform.location.x} - {wp1.transform.location.y}) - ({wp2.transform.location.x} - {wp2.transform.location.y})")
            draw_couple(pair_counter, wp1, wp2)
            draw_route( wp1, wp2, pair_counter, 0, 0, 255)
        else:
            # altrimenti c'è un angolo tra i due wp
            #print(f"Pair {idx}: tra i waypoints c'è una curva ({wp1.transform.location.x} - {wp1.transform.location.y}) - ({wp2.transform.location.x} - {wp2.transform.location.y})")
            draw_couple(pair_counter, wp1, wp2)
            draw_route(wp1, wp2, pair_counter, 255, 255, 255 )
            
        pair_indexer[pair_counter] = (wp1, wp2);
        pair_counter = pair_counter + 1

##############################################      MAIN CODE      ############################################

##############Connessione al simulatore################
client = carla.Client('localhost', 2000)
client.set_timeout(5.0)
world = client.get_world()
bp_lib = world.get_blueprint_library() 
map = world.get_map()
#######################################################

# lista di wp alternati (es.: ogni 2 metri)
waypoints = map.generate_waypoints(2.0)

# array di incroci contenente coppie (id, oggetto_incrocio)
unique_junctions = []

# lettura dei wp eventualmente presenti ad un' intersezione
#N.B.: più waypoint possono appartenere a uno stesso incrocio
for waypoint in waypoints:
    if waypoint.is_junction:
        junction = waypoint.get_junction()
        if not(any(u[0] == junction.id for u in unique_junctions)):
            unique_junctions.append( (junction.id, junction))

# Stampa il numero di incroci (9 distinti in town 10)
print(f"Numero di incroci trovati: {len(unique_junctions)}\n")

#la prima cella è l'iesimo elemento dell'array fatto di coppie (id, intersezione)
#la seconda cella lasciala a 1
#la penultima le coppie e l'ultima il primo elemento di ciascuna coppia
wp_pairs = unique_junctions[1][1].get_waypoints(carla.LaneType.Driving)
wp1 = wp_pairs[4][0]


#per ora wp pairs è una lista presa a caso
grouped_pairs = group_pairs_by_second_waypoint(wp_pairs)

spawned_vehicles = []
# Stampa i gruppi di coppie per ciascun secondo waypoint
for wp2_coords, pairs in grouped_pairs.items():

    if len(pairs) > 1:
       draw_intersection(pairs)
       spawned_vehicles += (spb.spawn_all_behind_wp_couple(world, pairs, distance=10, focus=1))

print(spawned_vehicles)

# qui bisogna impostare l'agent per ogni veicolo
#prova a disattivare i semafori e a vedere come si comportano
for v,s in spawned_vehicles:
    if v != None:
        v.set_autopilot(True)


Numero di incroci trovati: 9

Le coordinate (-71.27, 132.315) accomunano le seguenti coppie:
(-48.483) - (114.965), (-71.27) - (132.315)
(-28.539) - (133.536), (-71.27) - (132.315)

Le coordinate (-70.49, 128.903) accomunano le seguenti coppie:
(-51.983) - (114.975), (-70.49) - (128.903)
(-28.517) - (130.036), (-70.49) - (128.903)

[(<carla.libcarla.Vehicle object at 0x000002C0114F8900>, <carla.libcarla.Transform object at 0x000002C013D42BC0>), (<carla.libcarla.Vehicle object at 0x000002C0114F9EE0>, <carla.libcarla.Transform object at 0x000002C013D2DA40>), (None, <carla.libcarla.Transform object at 0x000002C013D42AC0>), (<carla.libcarla.Vehicle object at 0x000002C0114F9E70>, <carla.libcarla.Transform object at 0x000002C013D2DAC0>)]
